In [ ]:
import pandas as pd
import numpy as np
import csv
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import neologdn
import demoji
import emoji

model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
# model_dir = '/Users/labimac/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)
mecab = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')


In [ ]:
path = "/Users/iomacbookair2/Documents/lab/DEIM2023/tweet_csv/nichiten/230115_nichiten.csv"
# path = "/Users/labimac/Documents/lab/DEIM2023/tweet_csv/221214_ann_wed.csv"

df = pd.read_csv((path))
df.sort_values(by = 'created_at', ascending = True, inplace = True)
df = df.reset_index(drop=True)
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.drop("author_id", axis=1)
df = df.drop("username", axis=1)
df = df.drop("tweet_id", axis=1)
df = df.drop("like_count", axis=1)
df = df.drop("retweet_count", axis=1)
df

In [ ]:
def preprocess(text):
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'#\S+', '', text) # ハッシュタグを消す
    text = re.sub(r'＃\S+', '', text) # ハッシュタグを消す
    text = re.sub(r'@\S+', '', text) # @を消す
    text = re.sub(r'http?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text) # URLをスペースに置き換え
    text = re.sub(r'https?://\S+', ' ', text) # URLをスペースに置き換え
    text = re.sub(r'!-/:-@[-`{-~]', r'', text) # 記号を消す
    text = re.sub(r'”', '', text)  # "を消す
    text = re.sub(r'"', '', text)  # "を消す
    text = re.sub(r"'", "", text) # 'を消す
    text = re.sub(r'[(-`)]', '', text)
    text = re.sub(
        "[\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\u3000-\u303F]", '', text)  # 記号を消す
    text = re.sub(
        u'[■-♯【】「」『』;・ㅂﾟˊᗜ┏━┓┗┛ヾθ、│!-/`:-@-`{-~。|∀!〇╰`′‵ˋω.*ー…╭∇^_￣´｀•˘д↑艸╯→°д̀ᴗ˃˂⁽⁾φ└＼※彡𖥦←ꂹ]', '', text)  # 記号を消す
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text) # 小数点とカンマを消す
    text = re.sub(r'\d+', '0', text) # 数字を0に置き換え
    text = text.lower() # 英字を小文字に
    text = re.sub(r"[\u3000\t\r\n]", " ", text) # 空白文字をスペースに置き換え
    text = neologdn.normalize(text)
    return text


df['text'] = df['text'].apply(preprocess)
df


In [ ]:
df.to_csv('220115_preprocessed.csv', index=False)

In [ ]:
def word_to_vec(sentence, model, mecab):
    if not sentence:
        sentence_embedding = np.zeros(200, dtype=np.float32)
    else:
        pre_sentence = sentence.split(" ")
        sentence_words = []
        for s in pre_sentence:
            sentence_words += [line.split("\t")[0]
                               for line in mecab.parse(s).split("\n")[:-2]]
        if all(re.match(r'.*[a-zA-Z].*', word) for word in sentence_words):
            sentence_embedding = np.zeros(200, dtype=np.float32)
        else:
            word_vectors = [model[word]
                            for word in sentence_words if word in model]
            if word_vectors:
                if len(word_vectors) == 0:
                    sentence_embedding = np.zeros(200, dtype=np.float32)
                else:
                    sentence_embedding = np.mean(word_vectors, axis=0)
            else:
                sentence_embedding = np.zeros(200, dtype=np.float32)
    return sentence_embedding

df["vector"] = df["text"].apply(
    lambda x: 0 if len(x) <= 1 else word_to_vec(x, model_word2vec, mecab))
df = df.drop("text", axis=1)
df


In [ ]:
df.to_csv('220115_vector.csv', index=False)

In [ ]:
groups = df.groupby(pd.Grouper(key='created_at', freq='min'))
df_vectors = pd.DataFrame({
    "vectors": groups.apply(lambda x: x["vector"].tolist())
})
df_vectors = df_vectors.reset_index()
df_vectors


In [ ]:
df_vectors.to_csv('220115_vector_list.csv', index=False)

In [ ]:
def calc_similarity(vector1, vector2):
  similarity = cosine_similarity(
      vector1, vector1, vector2)[0][0]
  return similarity


def calc_average_similarity(vectors, key):
    similarities = []
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            print(f'i={i},j={j}')
            if np.all(vectors[i] == 0) or np.all(vectors[j] == 0):
                similarity = 0
            else:
                similarity = calc_similarity(vectors[i].reshape(1, -1), vectors[j].reshape(1, -1))
            print(similarity)
            similarities.append(similarity)
    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"q{i}"] = np.quantile(similarities, q[i])
    outputs["standard_deviation"] = np.std(similarities)
    outputs["mean"] = np.mean(similarities)
    if key == "q2":
        return outputs["q2"]
    elif key == "q1":
        return outputs["q1"]
    elif key == "q3":
        return outputs["q3"]
    elif key == "standard_deviation":
        return outputs["standard_deviation"]
    elif key == "mean":
        return outputs["mean"]


In [ ]:
df_vectors["q1"] = df_vectors["vectors"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity(x, "q1"))


In [ ]:
df_vectors["q2"] = df_vectors["vectors"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity(x, "q2"))

In [ ]:
df_vectors["q3"] = df_vectors["vectors"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity(x, "q3"))


In [ ]:
df_vectors["stdev"] = df_vectors["vectors"].apply(
    lambda x: 0 if len(x) <= 1 else calc_average_similarity(x, "standard_deviation"))


In [ ]:
df_vectors = df_vectors.drop("vectors", axis=1)
df_vectors


In [ ]:
df_vectors.to_csv('220115_similarity.csv', index=False)


In [ ]:
# def calc_similarity_word2vec(sentence1, sentence2, model, mecab):
#   pre_sentence1 = sentence1.split(" ")
#   sentence1_words = []
#   for s1 in pre_sentence1:
#     sentence1_words += [line.split("\t")[0]
#                         for line in mecab.parse(s1).split("\n")[:-2]]
#   pre_sentence2 = sentence2.split(" ")
#   sentence2_words = []
#   for s2 in pre_sentence2:
#     sentence2_words += [line.split("\t")[0]
#                         for line in mecab.parse(s2).split("\n")[:-2]]
#   if not sentence1_words and not sentence2_words:
#     similarity = 1
#   elif not sentence1_words or not sentence2_words:
#     similarity = 0
#   elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
#     similarity = 0
#   elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
#     similarity = 0
#   else:
#     sentence1_embedding = np.mean([model[word]
#                                   for word in sentence1_words if word in model], axis=0)
#     sentence2_embedding = np.mean([model[word]
#                                   for word in sentence2_words if word in model], axis=0)
#     if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
#         similarity = 0
#     else:
#       similarity = cosine_similarity(
#           [sentence1_embedding], [sentence2_embedding])[0][0]
#   return similarity


# def calc_average_similarity_word2vec(sentences, key, mecab):
#     similarities = []
#     for i in range(len(sentences)):
#         for j in range(i + 1, len(sentences)):
#             similarity = calc_similarity_word2vec(
#                 sentences[i], sentences[j], model_word2vec, mecab)
#             similarities.append(similarity)
#     q = [0, 0.25, 0.5, 0.75, 1]
#     outputs = {}
#     for i in range(len(q)):
#         outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
#     outputs["q1"] = outputs["quantile_1"]
#     outputs["q2"] = outputs["quantile_2"]
#     outputs["q3"] = outputs["quantile_3"]
#     outputs["standard_deviation"] = np.std(similarities)
#     outputs["mean"] = np.mean(similarities)
#     if key == "q2":
#         return outputs["quantile_2"]
#     elif key == "q1":
#         return outputs["quantile_1"]
#     elif key == "q3":
#         return outputs["quantile_3"]
#     elif key == "standard_deviation":
#         return outputs["standard_deviation"]
#     elif key == "mean":
#         return outputs["mean"]
